In [2]:
import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [20]:
## Load dataset

IMG_SHAPE = (96, 96, 3)
data = pd.read_csv("updated_train.csv")

Inspection of the dataset.

In [21]:
print("Dimensions of the dataset: ", data.shape)
print(data.loc[0,:])

Dimensions of the dataset:  (7049, 31)
left_eye_center_x                                                      66.0336
left_eye_center_y                                                      39.0023
right_eye_center_x                                                      30.227
right_eye_center_y                                                     36.4217
left_eye_inner_corner_x                                                59.5821
left_eye_inner_corner_y                                                39.6474
left_eye_outer_corner_x                                                73.1303
left_eye_outer_corner_y                                                  39.97
right_eye_inner_corner_x                                               36.3566
right_eye_inner_corner_y                                               37.3894
right_eye_outer_corner_x                                               23.4529
right_eye_outer_corner_y                                               37.3894
left_eyebrow_

Let's find if there are missing values 

In [22]:
data.isnull().sum()/data.shape[0]

left_eye_center_x            0.001419
left_eye_center_y            0.001419
right_eye_center_x           0.001844
right_eye_center_y           0.001844
left_eye_inner_corner_x      0.671443
left_eye_inner_corner_y      0.671443
left_eye_outer_corner_x      0.672010
left_eye_outer_corner_y      0.672010
right_eye_inner_corner_x     0.672010
right_eye_inner_corner_y     0.672010
right_eye_outer_corner_x     0.672152
right_eye_outer_corner_y     0.672152
left_eyebrow_inner_end_x     0.671585
left_eyebrow_inner_end_y     0.671585
left_eyebrow_outer_end_x     0.678252
left_eyebrow_outer_end_y     0.678252
right_eyebrow_inner_end_x    0.671585
right_eyebrow_inner_end_y    0.671585
right_eyebrow_outer_end_x    0.676834
right_eyebrow_outer_end_y    0.676834
nose_tip_x                   0.000000
nose_tip_y                   0.000000
mouth_left_corner_x          0.671868
mouth_left_corner_y          0.671868
mouth_right_corner_x         0.671726
mouth_right_corner_y         0.671726
mouth_center

It looks like roughly half of the database is incomplete!
Let's try and find a way to fill up the missing data

## Prepare the dataset

In [38]:
from sklearn.model_selection import train_test_split

dftrain = data.dropna()
Y = dftrain.iloc[:, :-1]
X_raw = dftrain.iloc[:,-1]
dim_output = len(Y_raw.columns.values)
original_batch_size = len(X_raw)
print(X_raw.shape, Y.shape)

(2180,) (2180, 30)


In [39]:
def convert_X(Xdf):
    batch_size = len(Xdf)
    X = np.zeros((batch_size, 96, 96, 1))

    for row in range(batch_size):
        X[row, :,:, :] = np.array([float(i) for i in Xdf.iloc[row].split(" ")]).reshape(1,96,96,1)/255
    return X

XX = convert_X(X_raw)
XX_shape = list(XX.shape)
XX_shape[-1] = 3
X = np.zeros(XX_shape)
for i in range(3):
    X[:,:,:,i] = XX[:,:,:,0]

In [42]:
# Split train, validation and test set

XXtrain, Xtest, YYtrain, Ytest = train_test_split(X, Y, test_size = 0.05)
Xtrain, Xval, Ytrain, Yval = train_test_split(XXtrain, YYtrain, test_size = 0.2)

print("Train set: ", Xtrain.shape, Ytrain.shape)
print("Validation set: ", Xval.shape, Yval.shape)
print("Test set: ", Xtest.shape, Ytest.shape)

Train set:  (1656, 96, 96, 3) (1656, 30)
Validation set:  (415, 96, 96, 3) (415, 30)
Test set:  (109, 96, 96, 3) (109, 30)


## Apply transfer learning with pre-trained CNN
https://www.tensorflow.org/tutorials/images/transfer_learning?hl=it

In [24]:
import tensorflow as tf
keras = tf.keras
# Specify include_top=False not to include the last layer
base_model = keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                           include_top = False,
                                           weights = "imagenet")

9412608/9406464 [==============================] - 28s 3us/step


In [46]:
# Set the base model not to be trainable
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_96"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 97, 97, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 48, 48, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 48, 48, 32)   128         Conv1[0][0]                      
________________________________________________________________________________